# Test if features (mean and iq) are the spected values

In [ ]:
 from sklearn.metrics import f1_score

# optimizeraa
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.5)

# loss function
lossFunction = nn.CrossEntropyLoss()

# loss
train_loss = np.zeros((epochs,))
test_loss = np.zeros((epochs,))

# f1 scores
f1Scores = np.zeros((epochs, ))

# min global test loss 
minTestLossGlobalSoFar = float("inf")

# # # loss plot
# if it is not cluster
if (not trainingOnGuanaco) or (not trainWithJustPython):
    
    # add f1 and loss plots
    fig, ax = plt.subplots(1, 2, figsize = (7, 3), tight_layout = True)
    # # fig, ax = plt.subplots()
    
    # error
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Error")
    
    
    # f1 score
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel("F1 score")
    

# early stopping
# prior_test_error = 0
count_early_stop = 0
threshold_early_stop = threshold_early_stop


print("starting the training")


# epoch
for nepoch in range(epochs):
        
    print("epoch:    {0} / {1}".format(nepoch, epochs))
    
    
    
     
    ######## Train ###########
    epoch_train_loss = 0
    
    for data_ in trainLoader:
        
        data = data_[0]
        labels = data_[1].to(device = cuda_device)
#         labels = data_[1]
        
        optimizer.zero_grad()
            
        # this take the deltas (time and magnitude)
        data = generateDeltas(data, passband, includeDeltaErrors).type(torch.FloatTensor).to(device = cuda_device)
#         data = generateDeltas(data, passband).type(torch.FloatTensor)
            
        # add other features
        # [batch size, features]
#         if includeOtherFeatures:
        if includeOtherFeatures:
            
            otherFeatures = getOtherFeatures(data_[0]).to(device = cuda_device)
                
#             print(np.any(torch.isnan(otherFeatures).cpu().numpy()))
            
            if np.any(torch.isnan(otherFeatures).cpu().numpy()):
                
                print(f"other features with nan values in epoch {nepoch}")
            
            
            # get model output
            outputs = model.forward(data, includeDeltaErrors, otherFeatures)
            
            
            # #         # testing tensor size 
# #         assert data.shape == torch.Size([batch_training_size, len(passband), 4, 71]), "Shape should be [minibatch size, channels, 4, 71]"
# #         print("test ok")
        
        else:
            
            # get model output
            outputs = model.forward(data, includeDeltaErrors)

        
#         break

In [ ]:
# testing if means and iq are correct

channels = [0, 1, 2, 3, 4, 5]
indexs = np.random.choice(len(data_[2]), 2)

# print(indexs)

fig, ax = plt.subplots(
    len(indexs), 
    len(channels),
    figsize = (12, 2*len(indexs)),
    tight_layout = True,
)

for index in np.arange(len(indexs)):
    
    for channel in channels:
    
        mask = data_[0][indexs[index], channel, 3, :]

        mask = mask.type(torch.BoolTensor)
    
#         print(f"index: {indexs[index]}")
        
        # print("data shape")
        # print(data_[0].shape)

        # datashape: [ 128, 6, 4, 72 ]
        ax[index][channel].scatter(
            data_[0][indexs[index], channel, 0, mask], 
            data_[0][indexs[index], channel, 1, mask],
        )

        # print("values:")
        # print(otherFeatures[index, 0])


        # manual mean
        manualMean = torch.mean(data_[0][indexs[index], channel, 1, mask])

#         ax[index][channel].hlines(
#             manualMean, 
#             xmin = data_[0][indexs[index], channel, 0, mask][0], 
#             xmax = data_[0][indexs[index], channel, 0, mask][-1],
#             color = "r"
#         )
        
        # analyze features
        ax[index][channel].hlines(
            otherFeatures[indexs[index], channel].item(), 
            xmin = data_[0][indexs[index], channel, 0, mask][0], 
            xmax = data_[0][indexs[index], channel, 0, mask][-1],
            color = "r"
        )
        
        # test mean
        assert manualMean.item() == otherFeatures[indexs[index], channel].item()
        print("mean test ok")

        
        iqManual =  torch.kthvalue(data_[0][indexs[index], channel, 1, mask], int(0.75*data_[0][indexs[index], channel, 1, mask].shape[0]))[0] - torch.kthvalue(data_[0][indexs[index], channel, 1, mask], int(0.25*data_[0][indexs[index], channel, 1, mask].shape[0]))[0]

        assert iqManual.item() == otherFeatures[indexs[index], 6 + channel].item()
        print("iq test ok")
        
        
#         print(manualMean.item())
#         print(otherFeatures[indexs[index], channel].item())
#         print("----")

# Test if samplers generate same data in different epochs
This save ids of train and validation loader by epoch and then compare them (full arrays and by epoch).
- If ids are equal by epoch, so taht means the dataloader (and the sampler in the background) is generating always the same dataset (ids and the same order).

- If ids are not equals, that means that by epoch, the loader (and the sampler in the background) is generating different datasets (sampling generating different ids and orders)

In [1]:
from sklearn.metrics import f1_score

nepochs_ = 10

idsTrain0 = np.zeros((nepochs_, train_size))
idsVal0 = np.zeros((nepochs_, validation_size))

# epoch
for nepoch in range(nepochs_):
        
    print("epoch:    {0} / {1}".format(nepoch, epochs))
    
    
    
     
    ######## Train ###########
    
    
    
    # this is for getting the other features
    trainLastIndex = 0
    
    for idx, data_ in enumerate(trainLoader):
        
        trainLastIndex_ = trainLastIndex + data_[0].shape[0]
        
        idsTrain0[nepoch, trainLastIndex: trainLastIndex_] = data_[2]
        
#         print(trainLastIndex, trainLastIndex_)
        
        trainLastIndex = trainLastIndex_
        
     # this is for getting the other features
    trainLastIndex = 0
    
    for idx, data_ in enumerate(validationLoader):
        
        trainLastIndex_ = trainLastIndex + data_[0].shape[0]
        
        idsVal0[nepoch, trainLastIndex: trainLastIndex_] = data_[2]
        
#         print(trainLastIndex, trainLastIndex_)
        
        trainLastIndex = trainLastIndex_

NameError: name 'np' is not defined

In [ ]:
# test  ids arrays are not equals
assert not np.array_equal(idsTrain0, idsVal0)

In [ ]:
from sklearn.metrics import f1_score

nepochs_ = 10

idsTrain = np.zeros((nepochs_, train_size))
idsVal = np.zeros((nepochs_, validation_size))

# epoch
for nepoch in range(nepochs_):
        
    print("epoch:    {0} / {1}".format(nepoch, epochs))
    
    
    
     
    ######## Train ###########
    
    
    
    # this is for getting the other features
    trainLastIndex = 0
    
    for idx, data_ in enumerate(trainLoader):
        
        trainLastIndex_ = trainLastIndex + data_[0].shape[0]
        
        idsTrain[nepoch, trainLastIndex: trainLastIndex_] = data_[2]
        
#         print(trainLastIndex, trainLastIndex_)
        
        trainLastIndex = trainLastIndex_
        
     # this is for getting the other features
    trainLastIndex = 0
    
    for idx, data_ in enumerate(validationLoader):
        
        trainLastIndex_ = trainLastIndex + data_[0].shape[0]
        
        idsVal[nepoch, trainLastIndex: trainLastIndex_] = data_[2]
        
#         print(trainLastIndex, trainLastIndex_)
        
        trainLastIndex = trainLastIndex_

In [ ]:
# test  ids arrays are equals
assert np.array_equal(idsTrain0, idsTrain)

In [ ]:
# test  ids arrays are equals
assert np.array_equal(idsVal0, idsVal)

In [ ]:
# test  ids arrays are not equals
assert not np.array_equal(idsTrain, idsVal)

In [ ]:
# test ids are equals
for i in np.arange(nepochs_):
    
    # test train ids
    assert (np.array_equal(idsTrain[0, :], idsTrain[i, :]))
    
    # test validation ids
    assert (np.array_equal(idsVal[0, :], idsVal[i, :])), f"in epoch: {i}"